In [2]:
__PRODUCTION__ = 1
__NAME__       = 'manifold'
__WIDTH__      = 5.5  # NeurIPS 2021 text box width
__HEIGHT__     = 3.35

if __PRODUCTION__:
    from mplmagic2 import pgf
else:
    from mplmagic2 import svg

from mplmagic2 import SuperFigure, tex, colormap_apply
import matplotlib.pyplot as plt

import functools
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim
import pickle
from symfac.experimental import RBFExpansionV2
from symfac.experimental import RBFExpansionMiniBatch
from collections import OrderedDict
from functools import partial
from time import time

from sklearn import manifold, datasets

print('This is how much space the figure will take up on letter paper')
SuperFigure.size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [3]:
time_nn, learned_order_nn, data_nn = pickle.load(open('data/paper/manifold_00.pickle', "rb"))
time_02, learned_order_02, data_02 = pickle.load(open('data/paper/manifold_02.pickle', "rb"))
time_04, learned_order_04, data_04 = pickle.load(open('data/paper/manifold_04.pickle', "rb"))
time_06, learned_order_06, data_06 = pickle.load(open('data/paper/manifold_06.pickle', "rb"))

In [6]:
fig = SuperFigure(figsize=(__WIDTH__, __HEIGHT__), dpi=300)
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

title_style = dict(
    fontsize=8,
    va='top'
)

# draw the axes grid
x0 = 0.03
y0 = 0.09
dx = 0.245
dy = 0.48
w = 0.23
marginx = 0.02
marginy = 0.075
axs = np.array([
    [
        fig.make_axes(
            left=x0 + dx * i - marginx,
            width=w + marginx * 2,
            top=y0 - marginy,
            width_to_height=0.7,
            style='modern',
            projection='3d'
        ) for i in range(4)
    ],
    [
        fig.make_axes(
            left=x0 + dx * i,
            width=w,
            top=y0 + dy,
            width_to_height=1.0,
            style='modern'
        ) for i in range(4)
    ],
])


ticks_style = dict(
    fontsize=7,
)
_3d_scatter_style = dict(
    s=16 if __PRODUCTION__ else 1,
    marker='o',
    edgecolor='#202020',
    linewidth=0.5,
    cmap='Spectral',
    alpha=0.5,
)

for ax, data, color, noise in zip(
    axs[0],
    [data_nn, data_02, data_04, data_06],
    [time_nn, time_02, time_04, time_06],
    [0, 0.2, 0.4, 0.6]
):
    ax.patch.set_facecolor('none')
    ax.set_scale(zoom=(1, 1, 1), pan=(-0.015, 0, 0))
    ax.scatter(
        data[::2, 0],
        data[::2, 1],
        data[::2, 2],
        c=color[::2],
        **_3d_scatter_style
    )
    ax.set_xlim([-1.5, 1.5])
    ax.set_ylim([-2.5, 2.5])
    ax.set_zlim([-1.0, 2.0])
    xticks = [-1, 1]
    ax.set_xticks(xticks)
    ax.set_xticklabels([fr'{x:d}' for x in xticks])
    yticks = [-1, 1]
    ax.set_yticks(yticks)
    ax.set_yticklabels([fr'{y:d}' for y in yticks])
    ax.xaxis._axinfo['juggled'] = (0, 0, 0)
    ax.yaxis._axinfo['juggled'] = (0, 0, 0)
    ax.zaxis._axinfo['juggled'] = (0, 0, 0)
    ax.axis('off')
    ax.set_title(
        fr'$\mathcal{{N}}$(0, {noise:.1f})',
        y=1.14,
        **title_style
    )
    ax.xaxis.set_pane_color((1, 1, 1, 0))
    ax.yaxis.set_pane_color((1, 1, 1, 0))
    ax.zaxis.set_pane_color((1, 1, 1, 0))
    
    ax.grid(None)

def normalize(x):
    return (x - x.min())/(x.max() - x.min())

parity_scatter_style = dict(
    marker='o',
    # s=1.6 if __PRODUCTION__ else 0.1,
    s=1.6 if __PRODUCTION__ else 0.1,
    c=color,
    cmap=colormap_apply(
        'Spectral',
        lambda H, L, S: (H, L * 1.1, S * 1.2),
        mode='hls'
    ),
    edgecolor='#404040',
    lw=0.25,
)
axs[1, 0].scatter(normalize(time_nn), normalize(learned_order_nn), **parity_scatter_style)
axs[1, 1].scatter(normalize(time_02), normalize(learned_order_02), **parity_scatter_style)
axs[1, 2].scatter(normalize(time_04), normalize(learned_order_04), **parity_scatter_style)
axs[1, 3].scatter(normalize(time_06), normalize(learned_order_06), **parity_scatter_style)

tick_style = dict(
    fontsize=7,
)
xlabel_style = dict(
    fontsize=8,
    labelpad=-1.25 * (3 if __PRODUCTION__ else 1),
)
ylabel_style = dict(
    fontsize=8,
    labelpad=-1.0 * (2 if __PRODUCTION__ else 1),
)
for i, ax in enumerate(axs[1]):
    ticks = [0, 1]
    ax.set_xbound(lower=0, upper=1)
    ax.set_ybound(lower=0, upper=1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(['%d' % x for x in ticks], **tick_style)
    if i == 0:
        ax.set_yticklabels(['%d' % y for y in ticks], **tick_style)
        ax.set_ylabel(r'Learned embedding', **ylabel_style)
    else:
        ax.set_yticklabels([])
    ax.set_xlabel('Ground truth', **xlabel_style)

ax_overtitle = fig.make_axes(
    left=axs[1, 0].left,
    right=axs[1, -1].right,
    top=0,
    bottom=1,
    style='blank'
)
ax_overtitle.text(
    0.5, 0.99,
    r'\textbf{S-Curve with Additive Noise}',
    ha='center',
    **title_style
)
margin = 0.05
ax_overtitle.axhline(0.95, xmin=margin, xmax=1 - margin, lw=0.5, color='k')
 
if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)

In [7]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-manifold.pdf


END
---